In [90]:
import requests, pyperclip, base64, os, json

class Fitbit:
    
    auth_url = "https://www.fitbit.com/oauth2/authorize"
    token_url = "https://api.fitbit.com/oauth2/token"
    
    base_url = "https://api.fitbit.com"
    
    redirect_uri = "http://localhost"
    
    client_id = os.environ.get("fitbit_client_id")
    client_secret = os.environ.get("fitbit_client_secret")
    
    scope = ["activity", "nutrition", "heartrate", "location", "nutrition", "profile", "settings", "sleep", "social", "weight"]
    
    @staticmethod
    def copy_auth_url():
        req = requests.Request("GET", Fitbit.auth_url, params={
            "response_type": "code", "client_id": Fitbit.client_id, 
            "redirect_uri": Fitbit.redirect_uri, "scope": " ".join(Fitbit.scope)
        }).prepare()
        pyperclip.copy(req.url)
        
    @staticmethod
    def encoded_client():
        return base64.b64encode(f"{Fitbit.client_id}:{Fitbit.client_secret}".encode('ascii')).decode('ascii') 
            
    def __init__(self):
        self.client = Fitbit.encoded_client()
        Fitbit.copy_auth_url()
        self.authenticate(input("Auth code: "))
        
    def authenticate(self, auth_code):
        res = requests.post(Fitbit.token_url, 
            params={
                "code": auth_code, "grant_type": "authorization_code", 
                "client_id": Fitbit.client_id, "redirect_uri": Fitbit.redirect_uri}, 
            headers={
                "Authorization": f"Basic {self.client}", 
                "Content-Type": "application/x-www-form-urlencoded"})
        assert res.status_code == 200
        data = json.loads(res.text)
        self.user_id = data["user_id"]
        self.access_token = data["access_token"]
        self.refresh_token = data["refresh_token"]
        
    def __request(self, http_method, url, params):
        headers = { "Authorization": f"Bearer {self.access_token}" }
        res = http_method(f"{Fitbit.base_url}{url}",headers=headers,params=params)
        return json.loads(res.text)
    
    def __get(self, url, params={}):
        return self.__request(requests.get, url, params)
    
    def __post(self, url, params={}):
        return self.__request(requests.post, url, params)
    
    def __delete(self, url):
        return self.__request(requests.delete, url, params)
    
    """
    Activity
    
    Full documentation: 
      https://dev.fitbit.com/build/reference/web-api/activity/
    """
    
    def __get_user_activities(self, extension, params={}):
        return self.__get(f"/1/user/{self.user_id}/activities{extension}", params)
    
    def __post_user_activities(self, extension, params={}):
        return self.__post(f"/1/user/{self.user_id}/activities{extension}", params)
    
    def __delete_user_activities(self, extension, params={}):
        return self.__delete(f"/1/user/{self.user_id}/activities{extension}", params)
    
    def get_activity_summary_by_date(self, date):
        return self.__get_user_activities(f"/date/{date}.json")
    
    def log_activity(self, params):
        return self.__post_user_activities(".json", params)
    
    def get_lifetime_stats(self):
        return self.__get_user_activities(".json")
    
    def delete_activity_log(self, activity_log_id):
        return self.__delete_user_activities(f"/{activity_log_id}.json")
    
    def get_activity_log_list(self, params):
        return self.__get_user_activities("/list.json", params)
    
    def get_activity_tcx(self, log_id, include_partial_tcx=True):
        return self.__get_user_activities(f"/{log_id}.tcx")
    
    def browse_activity_types(self):
        return self.__get(f"/1/activities.json")
    
    def get_activity_type(self, activity_id):
        return self.__get(f"/1/activities/{activity_id}.json")
    
    def get_frequent_activities(self):
        return self.__get_user_activities(f"/frequent.json")
    
    def get_recent_activity_types(self):
        return self.__get_user_activities(f"/recent.json")
    
    def get_favorite_activities(self):
        return self.__get_user_activities(f"/favorite.json")
    
    def delete_favorite_activity(self, activity_id):
        return self.__delete_user_activities(f"/favorite/{activity_id}.json")
    
    def add_favorite_activity(self, activity_id):
        return self.__post_user_activities(f"favorite/{activity_id}.json")
    
    def get_activity_goals(self, period):
        return self.__get_user_activities(f"/goals/{period}.json")
    
    def update_activity_goals(self, period, type, value):
        return self.__post_user_activities(f"/goals/{period}.json",
            params={"type": type, "value": value})
    
    """
    Activity Intraday Time Series
    
    Full documentation:
      https://dev.fitbit.com/build/reference/web-api/activity/#get-activity-intraday-time-series
    """
    
    def get_activity_intraday_time_series(self):
        pass
    
    def get_intraday_time_series(self):
        pass
    
    def get_activity_intraday_time_series(self):
        pass
    
    def get_intraday_time_series(self):
        pass
    
    
    """
    Activity Time Series
    
    Full documentation: 
      https://dev.fitbit.com/build/reference/web-api/activity/#activity-time-series
    """
    
    def get_activity_resource_by_date_range(self):
        pass
    
    def get_activity_tracker_resource_by_date_range_time_series(self):
        pass
    
    def get_activity_time_series(self):
        pass
    
    """
    Auth
    
    Full documentation: 
      https://dev.fitbit.com/build/reference/web-api/oauth2/
    """
        
    def get_oauth2_access_token(self):
        pass
    
    def get_oauth2_introspect(self):
        pass
    
    """
    Body and Weight
    
    Full documentation: 
      https://dev.fitbit.com/build/reference/web-api/body/
    """
    
    def get_body_fat_logs_date(self):
        pass
    
    def get_body_fat_logs_period(self):
        pass
    
    def get_body_fat_logs_range(self):
        pass
    
    def log_body_fat(self):
        pass
    
    def delete_body_fat_log(self):
        pass
    
    def get_body_goals(self):
        pass
    
    def update_body_fat_goal(self):
        pass
    
    def update_weight_goal(self):
        pass
    
    def get_weight_logs(self):
        pass
    
    def log_weight(self):
        pass
    
    def delete_weight_log(self):
        pass
    
    """
    Body and Weight Time Series
    
    Full documentation: 
      https://dev.fitbit.com/build/reference/web-api/body/#body-time-series
    """
    def get_body_time_series_period(self):
        pass
    
    def get_body_time_series_range(self):
        pass
    
    """
    Devices
    
    Full documentation: 
      https://dev.fitbit.com/build/reference/web-api/devices/
    """
    def get_devices(self):
        pass
    
    def get_alarms(self):
        pass
    
    def add_alarm(self):
        pass
    
    def update_alarm(self):
        pass
    
    def delete_alarm(self):
        pass
    
    """
    Food and Water
    
    Full documentation: 
      https://dev.fitbit.com/build/reference/web-api/food-logging/
    """
    def get_food_locales(self):
        pass
    
    def get_food_goals(self):
        pass
    
    def update_food_goal(self):
        pass
    
    def get_food_logs(self):
        pass
    
    def get_water_logs(self):
        pass
    
    def get_water_goal(self):
        pass
    
    def update_water_goal(self):
        pass
    
    def log_food(self):
        pass
    
    def delete_food_log(self):
        pass
    
    def edit_food_log(self):
        pass
    
    def log_water(self):
        pass
    
    def delete_water_log(self):
        pass
    
    def update_water_log(self):
        pass
    
    def get_favorite_foods(self):
        pass
    
    def get_frequent_foods(self):
        pass
    
    def add_favorite_food(self):
        pass
    
    def delete_favorite_food(self):
        pass
    
    def get_meals(self):
        pass
    
    def create_meal(self):
        pass
    
    def edit_meal(self):
        pass
    
    def delete_meal(self):
        pass
    
    def get_recent_foods(self):
        pass
    
    def create_food(self):
        pass
    
    def delete_custom_food(self):
        pass
    
    def get_food(self):
        pass
    
    def get_food_units(self):
        pass
    
    def search_foods(self):
        pass
    

SyntaxError: unexpected EOF while parsing (<ipython-input-90-27f6a954edd0>, line 322)

In [87]:
fitbit = Fitbit()

Auth code: 84f35c0a5f878f2f8e000fb8537cdc0570a3547a


In [50]:
import pandas as pd

In [88]:
fitbit.get_frequent_activities()

[{'activityId': 90013,
  'calories': 0,
  'description': 'Walking less than 2 mph, strolling very slowly',
  'distance': 0,
  'duration': 1331000,
  'name': 'Walk'},
 {'activityId': 90009,
  'calories': 0,
  'description': 'Running - 5 mph (12 min/mile)',
  'distance': 1.61,
  'duration': 5580000,
  'name': 'Run'},
 {'activityId': 15000,
  'calories': 0,
  'description': '',
  'distance': 0,
  'duration': 1946000,
  'name': 'Sport'},
 {'activityId': 20047,
  'calories': 0,
  'description': '',
  'distance': 0,
  'duration': 2150000,
  'name': 'Elliptical'},
 {'activityId': 3001,
  'calories': 0,
  'description': '',
  'distance': 0,
  'duration': 1331000,
  'name': 'Aerobic Workout'},
 {'activityId': 1071,
  'calories': 0,
  'description': '',
  'distance': 0,
  'duration': 1587000,
  'name': 'Outdoor Bike'}]

In [73]:
data

{'activities-floors': [{'dateTime': '2019-01-01', 'value': '0'},
  {'dateTime': '2019-01-02', 'value': '0'},
  {'dateTime': '2019-01-03', 'value': '0'},
  {'dateTime': '2019-01-04', 'value': '0'},
  {'dateTime': '2019-01-05', 'value': '0'},
  {'dateTime': '2019-01-06', 'value': '0'},
  {'dateTime': '2019-01-07', 'value': '0'},
  {'dateTime': '2019-01-08', 'value': '0'},
  {'dateTime': '2019-01-09', 'value': '0'},
  {'dateTime': '2019-01-10', 'value': '0'},
  {'dateTime': '2019-01-11', 'value': '0'},
  {'dateTime': '2019-01-12', 'value': '0'},
  {'dateTime': '2019-01-13', 'value': '0'},
  {'dateTime': '2019-01-14', 'value': '0'},
  {'dateTime': '2019-01-15', 'value': '0'},
  {'dateTime': '2019-01-16', 'value': '0'},
  {'dateTime': '2019-01-17', 'value': '0'},
  {'dateTime': '2019-01-18', 'value': '0'},
  {'dateTime': '2019-01-19', 'value': '0'},
  {'dateTime': '2019-01-20', 'value': '0'},
  {'dateTime': '2019-01-21', 'value': '0'},
  {'dateTime': '2019-01-22', 'value': '0'},
  {'dateTim

In [65]:
df = pd.DataFrame(data=data['activities-steps'])
df['value'] = df['value'].apply(lambda x: int(x))

In [69]:
dfFilter = df[df['value'] != 0]
sum(dfFilter['value']) / len(dfFilter)

10585.510466988728